# Imports

In [9]:
import config
import requests

In [10]:
import riotwatcher
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Helper Functions

In [11]:
def check_rate(rate, limit):
    rate +=1
    if rate > limit-5:
        time.sleep(120)
        rate = 0
    return rate

In [12]:
def scrape_player_data(size, depth, verbose=True):
    """
    Scrapes from an intial summoner, gathering summoner puuIDs in a set (to avoid duplicates)
    until set is equal to size in size.
    
    LOGIC:
    GET INIT
    GET MATCH
    GET PARTICIPANTS
    ADD UNIQUE PARTICIPANTS TO UNIQUE_CHAIN
    ADD PARTICIPANTS TO SUMMONER_SET
    FROM UNIQUE PARTICIPANTS GET MATCH
    GET PARTICIPANTS
    """
    

    #Sets up API
    tft = riotwatcher.TftWatcher(config.api_key)
    #GET INIT
    initial_summoner = tft.summoner.by_name('na1', config.initial_summoner)
    
    #Return object
    summoner_list = [initial_summoner['puuid']]
    
    #Current Summoner is puuID
    current_summoner = summoner_list[-1]
    
    while(len(summoner_list) < size):
        
        #counts number of requests made to API
        #limit is 100/2min
        rate = 0
        
        current_match_id_list = tft.match.by_puuid('americas',current_summoner, count=depth)
        
        rate = check_rate(rate, 100)
        
        for current_match_id in current_match_id_list:
            current_match = tft.match.by_id('americas',current_match_id)
            
            rate = check_rate(rate,100)
            
            for participant_puuid in current_match['metadata']['participants']:
                
                #New puuID
                if participant_puuid not in summoner_list:
                    summoner_list.append(participant_puuid)
                    
                    if verbose:
                        if len(summoner_list) % 1000 == 0:
                            if verbose:
                                print("*" * 30)
                                print(f'Reached {len(summoner_list)} Rows')
                                print(f'Current Time: [{time.strftime("%H:%M:", time.localtime())}]')
                                
                
        if current_summoner == summoner_list[-1]:
            current_summoner = summoner_list[summoner_list.index(current_summoner)-1]
        
        else:
            current_summoner = summoner_list[-1]
    
    return summoner_list

In [13]:
def scrape_match_data(puuid_list, matches_per_player, verbose=True):
    
    """
    Taking in a list of puuid's, scrape_match_data retrieves the [matches_per_player] most recent matches.
    Stores them all in an empty DataFrame
    And Returns DataFrame
    """
    
    res = pd.DataFrame()
    rate = 0
    
    for puuid in puuid_list:
        try:
            match_list = tft.match.by_puuid('americas', puuid, count=matches_per_player)
            rate = check_rate(rate,200)
            for match_id in match_list:
                match=tft.match.by_id('americas', match_id)
                rate =check_rate(rate,200)
                #Version Check
                if 'Version 12.3' in match['info']['game_version']:
                    #Double Up Check
                    try:
                        match['info']['participants'][0]['partner_group_id']
                    #Not Double Up
                    except:
                        match_df = pd.concat({match_id:pd.DataFrame(match['info']['participants'])}, names=['MATCH_ID'])
                        res = pd.concat([res,match_df])                
                #Version Mismatch
                else: 
                    continue
                if res.shape[0] % 1000 == 0:
                    if verbose:
                        print("*" * 30)
                        print(f'Reached {res.shape[0]} Rows')
                        print('Saving...')
                        print(f'Current Time: [{time.strftime("%H:%M:", time.localtime())}]')
                    res.to_csv('../datasets/match_df.csv', index_label=['MATCH_ID','index'])
        except requests.exceptions.HTTPError as exception:
            if verbose:
                print('*' * 30)
                print(f'Recieved Error: {exception.response.status_code}')
                print(f'Waiting 5 Minutes Before Continuing')
                print(f'Current Time: [{time.strftime("%H:%M:", time.localtime())}]')
            time.sleep(300)
            
    return res

# API/Wrapper Setup

In [15]:
tft = riotwatcher.TftWatcher(config.api_key)

In [8]:
initial_summoner = tft.summoner.by_name('na1', config.initial_summoner)

In [9]:
initial_summoner

{'id': '0J1bOAXph4ejEEIUOCksTA-zPXYaH5umfk3keQoLnpGcKLk',
 'accountId': 'kN0PWD_8PZ_3g287-X8kT3K99aP3QqWlY_aHimwQkAggyFA',
 'puuid': '1VowMpU3PoNpxMmVx67I3vQ14hC6GVIpz2n2TFq63OXvY26MnaVPFEgzfxySlVFOgemLIIEMYx4ZKA',
 'name': 'Kurrybo',
 'profileIconId': 1381,
 'revisionDate': 1643870198011,
 'summonerLevel': 344}

In [12]:
match_list = tft.match.by_puuid('americas', initial_summoner['puuid'], count=5)

In [13]:
match_list

['NA1_4204279123',
 'NA1_4203444536',
 'NA1_4198540090',
 'NA1_4196570920',
 'NA1_4196395629']

In [23]:
double_match = tft.match.by_id('americas', match_list[0])

In [24]:
double_match['info']['game_version']

'Version 12.3.421.3734 (Feb 01 2022/14:32:37) [PUBLIC] <Releases/12.3>'

In [29]:
double_match['info']['participants'][0]['partner_group_id']

1

In [20]:
solo_match = tft.match.by_id('americas', match_list[1])

In [21]:
solo_match['info']['game_version']

'Version 12.3.420.4308 (Jan 26 2022/16:46:06) [PUBLIC] <Releases/12.3>'

In [33]:
#solo_match['info']['participants'][0]['partner_group_id']

# Data Acquisition

## Summoner Puuid Scraping

In [6]:
player_list = scrape_player_data(10_000, 5) 

******************************
Reached 1000 Rows
Current Time: [13:59:]
******************************
Reached 2000 Rows
Current Time: [14:04:]
******************************
Reached 3000 Rows
Current Time: [14:09:]
******************************
Reached 4000 Rows
Current Time: [14:13:]
******************************
Reached 5000 Rows
Current Time: [14:17:]
******************************
Reached 6000 Rows
Current Time: [14:21:]
******************************
Reached 7000 Rows
Current Time: [14:25:]
******************************
Reached 8000 Rows
Current Time: [14:29:]
******************************
Reached 9000 Rows
Current Time: [14:33:]
******************************
Reached 10000 Rows
Current Time: [14:37:]


In [7]:
player_list.to_csv('../datasets/summoner_puuids.csv',index=False)

AttributeError: 'list' object has no attribute 'to_csv'

## Match Scraping

In [ ]:
player_list = pd.read_csv('../datasets/summoner_puuids.csv')

In [ ]:
player_list.head()

In [ ]:
match_df = scrape_match_data(player_list, 10)

******************************
Reached 0 Rows
Saving...
Current Time: [18:15:]
******************************
Reached 0 Rows
Saving...
Current Time: [18:15:]
******************************
Reached 1000 Rows
Saving...
Current Time: [18:24:]
******************************
Reached 2000 Rows
Saving...
Current Time: [18:30:]
******************************
Reached 3000 Rows
Saving...
Current Time: [18:35:]
******************************
Reached 4000 Rows
Saving...
Current Time: [18:40:]
******************************
Reached 5000 Rows
Saving...
Current Time: [18:44:]
******************************
Reached 6000 Rows
Saving...
Current Time: [18:47:]
******************************
Reached 7000 Rows
Saving...
Current Time: [18:51:]
******************************
Reached 8000 Rows
Saving...
Current Time: [18:56:]
******************************
Reached 9000 Rows
Saving...
Current Time: [19:01:]
******************************
Reached 10000 Rows
Saving...
Current Time: [19:07:]
*********************